In [1]:
import torch
from datasets.citispace import create_cityscapes_dataloaders
import matplotlib.pyplot as plt
import numpy as np
import torch
from unet import UNet
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import CrossEntropyLoss
from train import train_one_epoch, validate
import json

In [ ]:
# Load the data
base_path = 'data'  # base path of where the cityscapes dataset is stored
train_loader, val_loader, test_loader = create_cityscapes_dataloaders(
    base_path=base_path,
    batch_size=8,
    num_workers=4
)

In [ ]:
# Get a batch of data and print sizes
images, masks = next(iter(train_loader))
print(f"Image size: {images.size()}")
print(f"Mask size: {masks.size()}")

In [ ]:
# Get Cityscapes classes from the dataset
CITYSCAPES_CLASSES = {
    0: 'background',
    1: 'road',
    2: 'sidewalk',
    3: 'building',
    4: 'wall',
    5: 'fence',
    6: 'pole',
    7: 'traffic light',
    8: 'traffic sign',
    9: 'vegetation',
    10: 'terrain',
    11: 'sky',
    12: 'person',
    13: 'rider',
    14: 'car',
    15: 'truck',
    16: 'bus',
    17: 'train',
    18: 'motorcycle',
    19: 'bicycle',
    255: 'ignore'  # Special ignore class
}

num_classes = 20  # Excluding ignore class (255)

# Create a color palette for visualization
COLOR_PALETTE = {
    0: (0, 0, 0),        # background: black
    1: (128, 64, 128),   # road: purple
    2: (244, 35, 232),   # sidewalk: pink
    3: (70, 70, 70),     # building: dark gray
    4: (102, 102, 156),  # wall: dark blue-gray
    5: (190, 153, 153),  # fence: light pink
    6: (153, 153, 153),  # pole: gray
    7: (250, 170, 30),   # traffic light: orange
    8: (220, 220, 0),    # traffic sign: yellow
    9: (107, 142, 35),   # vegetation: green
    10: (152, 251, 152), # terrain: light green
    11: (70, 130, 180),  # sky: blue
    12: (220, 20, 60),   # person: red
    13: (255, 0, 0),     # rider: bright red
    14: (0, 0, 142),     # car: dark blue
    15: (0, 0, 70),      # truck: darker blue
    16: (0, 60, 100),    # bus: medium blue
    17: (0, 80, 100),    # train: blue
    18: (0, 0, 230),     # motorcycle: bright blue
    19: (119, 11, 32),   # bicycle: dark red
    255: (0, 0, 0)       # ignore: black
}

# Function to plot colormap legend
def plot_colormap_legend():
    num_cols = 5
    num_rows = (len(CITYSCAPES_CLASSES) + num_cols - 1) // num_cols
    fig, ax = plt.subplots(figsize=(15, num_rows * 2))

    for idx, (class_id, class_name) in enumerate(CITYSCAPES_CLASSES.items()):
        if class_id == 255:  # Skip ignore class in visualization
            continue
        row = idx // num_cols
        col = idx % num_cols
        color = [x/255 for x in COLOR_PALETTE[class_id]]
        ax.add_patch(plt.Rectangle((col, -row), 1, 1, facecolor=color))
        ax.text(col + 0.5, -row - 0.5, class_name,
                ha='center', va='center', rotation=45)

    ax.set_xlim(0, num_cols)
    ax.set_ylim(-num_rows, 1)
    ax.axis('off')
    plt.title('Cityscapes Classes Color Map')
    plt.tight_layout()
    plt.show()


In [ ]:
# Function to plot images and masks
def plot_predictions(images, masks, predictions=None, n_samples=4):
    n_rows = 2 if predictions is None else 3
    fig, axes = plt.subplots(n_rows, n_samples, figsize=(15, 5*n_rows))

    for i in range(n_samples):
        # Plot original image
        img = images[i].permute(1, 2, 0).cpu().numpy()
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        axes[0, i].imshow(img)
        axes[0, i].axis('off')
        axes[0, i].set_title('Image')

        # Plot ground truth mask
        mask = masks[i].cpu().numpy()
        colored_mask = np.zeros((*mask.shape, 3))
        for class_id, color in COLOR_PALETTE.items():
            colored_mask[mask == class_id] = [x/255 for x in color]
        axes[1, i].imshow(colored_mask)
        axes[1, i].axis('off')
        axes[1, i].set_title('Ground Truth')

        # Plot prediction if available
        if predictions is not None:
            pred = torch.argmax(predictions[i], dim=0).cpu().numpy()
            colored_pred = np.zeros((*pred.shape, 3))
            for class_id, color in COLOR_PALETTE.items():
                colored_pred[pred == class_id] = [x/255 for x in color]
            axes[2, i].imshow(colored_pred)
            axes[2, i].axis('off')
            axes[2, i].set_title('Prediction')

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot the colormap legend and some sample images
plot_colormap_legend()
plot_predictions(images, masks)

In [ ]:
# https://stackoverflow.com/questions/71998978/early-stopping-in-pytorch

class EarlyStopping:
    """Early stopping callback similar to Keras implementation"""
    def __init__(
        self,
        monitor='val_loss',
        min_delta=0.0,
        patience=5,
        verbose=1,
        mode='min',
        restore_best_weights=True
    ):
        self.monitor = monitor
        self.min_delta = min_delta
        self.patience = patience
        self.verbose = verbose
        self.mode = mode
        self.restore_best_weights = restore_best_weights

        self.best_weights = None
        self.wait = 0
        self.stopped_epoch = 0
        self.best = float('inf') if mode == 'min' else float('-inf')

    def __call__(self, epoch, model, current):
        if self.mode == 'min':
            improved = current < (self.best - self.min_delta)
        else:
            improved = current > (self.best + self.min_delta)

        if improved:
            self.best = current
            self.wait = 0
            if self.restore_best_weights:
                self.best_weights = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                if self.verbose > 0:
                    print(f'Early stopping triggered at epoch {epoch}')
                if self.restore_best_weights and self.best_weights is not None:
                    if self.verbose > 0:
                        print('Restoring best weights')
                    model.load_state_dict(self.best_weights)
                return True
        return False


In [ ]:

# Training setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = UNet(n_channels=3, n_classes=num_classes).to(device)
criterion = CrossEntropyLoss(ignore_index=255)  # Ignore index for Cityscapes
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=5,
    min_lr=1e-6
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.003,
    patience=5,
    verbose=1,
    mode='min',
    restore_best_weights=True
)
# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'val_iou': [],
    'learning_rates': []
}

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    # Training phase
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, num_classes)

    # Validation phase
    val_loss, val_metrics = validate(model, val_loader, criterion, device, num_classes)

    # Update learning rate
    current_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_loss)

    # Store history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_iou'].append(val_metrics['iou'])
    history['learning_rates'].append(current_lr)

    # Print metrics
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Val IoU: {val_metrics['iou']:.4f}")
    print(f"Learning Rate: {current_lr:.6f}")

    # Check early stopping
    if early_stopping(epoch, model, val_loss):
        print(f"\nTraining stopped early at epoch {epoch+1}")
        break

# Save the final model and history
torch.save(model.state_dict(), 'cityscapes_ce_final_model.pth')
with open('cityscapes_ce_training_history.json', 'w') as f:
    json.dump(history, f)

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history['val_iou'], label='Val IoU')
plt.xlabel('Epoch')
plt.ylabel('IoU')
plt.legend()
plt.title('Validation IoU')

plt.tight_layout()
plt.show()

In [ ]:
# Load the best model and run predictions
import torch
from unet import UNet
import matplotlib.pyplot as plt
import numpy as np

def plot_predictions(images, masks, predictions, n_samples=4):
    """
    Plot images, ground truth masks, and predicted masks side by side
    Args:
        images: Tensor of shape (B, C, H, W)
        masks: Tensor of shape (B, H, W)
        predictions: Tensor of shape (B, C, H, W)
        n_samples: Number of samples to display
    """
    # Get predictions
    pred_masks = torch.argmax(predictions, dim=1)

    fig, axes = plt.subplots(n_samples, 3, figsize=(15, 5*n_samples))

    for i in range(n_samples):
        # Plot original image
        img = images[i].permute(1, 2, 0).cpu().numpy()
        # Denormalize image
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        axes[i, 0].imshow(img)
        axes[i, 0].axis('off')
        axes[i, 0].set_title('Image')

        # Plot ground truth mask
        mask = masks[i].cpu().numpy()
        colored_mask = np.zeros((*mask.shape, 3))
        for class_id, color in COLOR_PALETTE.items():
            if class_id != 255:  # Skip ignore label
                colored_mask[mask == class_id] = [x/255 for x in color]
        axes[i, 1].imshow(colored_mask)
        axes[i, 1].axis('off')
        axes[i, 1].set_title('Ground Truth')

        # Plot predicted mask
        pred_mask = pred_masks[i].cpu().numpy()
        colored_pred = np.zeros((*pred_mask.shape, 3))
        for class_id, color in COLOR_PALETTE.items():
            if class_id != 255:  # Skip ignore label
                colored_pred[pred_mask == class_id] = [x/255 for x in color]
        axes[i, 2].imshow(colored_pred)
        axes[i, 2].axis('off')
        axes[i, 2].set_title('Prediction')

    plt.tight_layout()
    plt.show()

def get_batch_predictions(model, data_loader, num_batches=1):
    """
    Get predictions for a few batches from the data loader
    """
    model.eval()
    all_images = []
    all_masks = []
    all_predictions = []

    with torch.no_grad():
        for i, (images, masks) in enumerate(data_loader):
            if i >= num_batches:
                break

            images = images.to(device)
            predictions = model(images)

            all_images.append(images)
            all_masks.append(masks)
            all_predictions.append(predictions)

    return (torch.cat(all_images, 0),
            torch.cat(all_masks, 0),
            torch.cat(all_predictions, 0))

# Load the best model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
loaded_model = UNet(n_channels=3, n_classes=num_classes).to(device)
loaded_model.load_state_dict(torch.load('cityscapes_ce_final_model.pth'))
loaded_model.eval()

# Get predictions for a few test batches
test_images, test_masks, test_predictions = get_batch_predictions(loaded_model, test_loader, num_batches=1)

# Plot the results
plot_predictions(test_images, test_masks, test_predictions, n_samples=4)

# Calculate and print metrics for the test set
criterion = CrossEntropyLoss(ignore_index=255)
test_loss, test_metrics = validate(loaded_model, test_loader, criterion, device, num_classes)
print("\nTest Set Metrics:")
print(f"Loss: {test_loss:.4f}")
print(f"IoU: {test_metrics['iou']:.4f}")
print(f"Dice Score: {test_metrics['dice']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")

# Print per-class IoU scores
print("\nPer-class IoU scores:")
for class_id, class_name in CITYSCAPES_CLASSES.items():
    if class_id != 255:  # Skip ignore class
        iou_key = f'class_{class_id}_iou'
        if iou_key in test_metrics:
            print(f"{class_name}: {test_metrics[iou_key]:.4f}")

# Optional: Create a confusion matrix visualization
import seaborn as sns

def plot_confusion_matrix(metrics):
    plt.figure(figsize=(15, 15))
    conf_matrix = metrics['confusion_matrix']
    #